### Строим модель на Keras

In [17]:
import splitfolders
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.optimizers import RMSprop
from keras import backend
import random
import scipy
from keras.utils import load_img
from keras.utils import img_to_array
from keras.models import load_model

In [8]:
train_data_dir = 'dataset_splited/train'
validation_data_dir = 'dataset_splited/val'

epochs = 50
batch_size = 20
img_width, img_height = 116, 116


if backend.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3) #116,116,3

In [9]:
train_datagen = ImageDataGenerator(
	rescale=1. / 255,
	shear_range=0.2,
	zoom_range=0.2,
	horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
	train_data_dir,
	target_size=(img_width, img_height),
	batch_size=batch_size,
	class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
	validation_data_dir,
	target_size=(img_width, img_height),
	batch_size=batch_size,
	class_mode='binary')

Found 30123 images belonging to 2 classes.
Found 7532 images belonging to 2 classes.


In [11]:
model = Sequential()
model.add(Conv2D(32, (2, 2), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
 
model.add(Conv2D(64, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
 
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.summary()

model.compile(loss='binary_crossentropy',
              optimizer = RMSprop(learning_rate=0.001),
              metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 115, 115, 32)      416       
                                                                 
 activation_5 (Activation)   (None, 115, 115, 32)      0         
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 57, 57, 32)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 56, 56, 32)        4128      
                                                                 
 activation_6 (Activation)   (None, 56, 56, 32)        0         
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 28, 28, 32)       0         
 2D)                                                  

In [13]:
model.fit(
	train_generator,
	steps_per_epoch=100,
	epochs=epochs,
	validation_data=validation_generator,
	validation_steps=50,
	verbose=-1)

model.evaluate(validation_generator)

Epoch 1/50
Epoch 2/50
Epoch 3/50
Epoch 4/50
Epoch 5/50
Epoch 6/50
Epoch 7/50
Epoch 8/50
Epoch 9/50
Epoch 10/50
Epoch 11/50
Epoch 12/50
Epoch 13/50
Epoch 14/50
Epoch 15/50
Epoch 16/50
Epoch 17/50
Epoch 18/50
Epoch 19/50
Epoch 20/50
Epoch 21/50
Epoch 22/50
Epoch 23/50
Epoch 24/50
Epoch 25/50
Epoch 26/50
Epoch 27/50
Epoch 28/50
Epoch 29/50
Epoch 30/50
Epoch 31/50
Epoch 32/50
Epoch 33/50
Epoch 34/50
Epoch 35/50
Epoch 36/50
Epoch 37/50
Epoch 38/50
Epoch 39/50
Epoch 40/50
Epoch 41/50
Epoch 42/50
Epoch 43/50
Epoch 44/50
Epoch 45/50
Epoch 46/50
Epoch 47/50
Epoch 48/50
Epoch 49/50
Epoch 50/50
377/377 [==============================] - 19s 51ms/step - loss: 0.0067 - accuracy: 0.9987


[0.0066898055374622345, 0.9986723065376282]

In [16]:
model.save("model_keras.h5")

### Тестируем модель

In [18]:
model = load_model('model_keras.h5')

def prediction(path):
    image = load_img(path, target_size=(116, 116))
    img_pred = img_to_array(image)
    img_pred = np.expand_dims(img_pred, axis=0)
    rslt = model.predict(img_pred)

    if rslt[0][0] == 1:
        prediction = 'рабочее место пустует'
    else:
        prediction = 'человек на рабочим месте'

    print(prediction)

In [20]:
prediction("dataset_splited\\val\\no_human\\crop718.jpg")

1/1 [==============================] - 0s 29ms/step
рабочее место пустует


In [21]:
prediction('dataset_splited\\val\\human\\crop22928.jpg')

1/1 [==============================] - 0s 35ms/step
человек на рабочим месте
